# **Network analysis - first part**

In this notebook, we will create networks between the individuals by their link to the same university at the same period. Then, conversely, we will realize a network between the organizations by the same individual.

The relations of the networks are created in an SQLite database. This needs many resources. We will realize in two parts. In this first part, we will only create the networks and in the **[second part](https://github.com/Semantic-Data-for-Humanities/Economists_Jurists/blob/main/Notebooks/Analysis_network_educatedAt_Script_part2.ipynb)**, we will make the analysis. Otherwise, this would exceed the number of bits allowed on Github.

Documentation:

* [Graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics) (Wikipedia en)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import sqlite3

In [138]:
import pprint
import csv
import ast
import re

from collections import Counter, defaultdict
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

import math

In [3]:
net = pd.read_csv("spreadsheets/Network/network_etucatedAt.csv", sep=",")
net=net.rename(columns={"year": "year_birth"})
net.head()

,person,name,year_birth,educ_id,educatedAt,longitude,latitude
0,urn:uuid:fb3527da-478c-4e4f-a861-9de00b9d154b,Stephen Harper,1959.0,http://www.wikidata.org/entity/Q1067471,University of Calgary,-114.133056,51.077500
1,urn:uuid:9ce56c33-08da-497c-afb0-38cb6ff6536c,Joseph C. O'Mahoney,1884.0,http://www.wikidata.org/entity/Q49088,Columbia University,-73.961944,40.807500
2,urn:uuid:9ce56c33-08da-497c-afb0-38cb6ff6536c,Joseph C. O'Mahoney,1884.0,http://www.wikidata.org/entity/Q846101,Georgetown University Law Center,-77.012500,38.898333
3,urn:uuid:490ab506-b377-4e39-bf39-3d6e3f732a62,"Hugh Meade Alcorn, Jr.",1907.0,http://www.wikidata.org/entity/Q1143289,Yale Law School,-72.928147,41.311581
4,urn:uuid:490ab506-b377-4e39-bf39-3d6e3f732a62,"Hugh Meade Alcorn, Jr.",1907.0,http://www.wikidata.org/entity/Q49116,Dartmouth College,-72.288333,43.703333


In [8]:
# transform dataframe into SQLite database for create a relation between two persons
# It is necessary to realise selfjoin with condition
# cf. https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
conn = sqlite3.connect(':memory:')
net.to_sql('net', conn, index=False)

In [5]:
# It is a query with the creation of relations between two people. 
# Both individuals are related if they went to the same higher education facility at the same period. 
# The period of the individuals is based on their date of birth. 
# It would be better to take the first and last of their university education (but few of them are recorded in Wikidata).
qry = '''
SELECT         net1.name AS person_1,
               net2.name AS person_2,
               net1.year_birth AS year_1,
               net2.year_birth AS year_2,
               net1.educatedAt AS educatedAt,
               net1.longitude AS longitude,
               net1.latitude AS latitude
FROM net net1, net net2
WHERE net1.name != net2.name
AND net1.educatedAt = net2.educatedAt
AND net1.year_birth BETWEEN net2.year_birth-6 AND net2.year_birth+6
AND net1.longitude = net2.longitude
AND net1.latitude = net2.latitude
    '''

In [6]:
# Transform the query result into dataframe
df = pd.read_sql_query(qry, conn)

In [7]:
df.to_csv("spreadsheets/network_educatedAt.csv", sep="|")

## **Link between organizations**

Now, we will realize the same process with the organization. A relation is created when an individual has been in two organizations.

In [77]:
link_edu = '''
SELECT         net1.educatedAt AS orga_1,
               net2.educatedAt AS orga_2,
               net1.name AS person,
               net1.year_birth AS birth_year,
               net1.longitude AS long_1,
               net2.longitude AS long_2,
               net1.latitude AS lat_1,
               net2.latitude AS lat_2
FROM net net1, net net2
WHERE net1.name = net2.name
AND net1.educatedAt != net2.educatedAt
AND net1.longitude != net2.longitude
AND net1.latitude != net2.latitude
ORDER BY person
    '''

In [78]:
df = pd.read_sql_query(link_edu, conn)

In [79]:
df.to_csv("spreadsheets/Network/network_organizations.csv", sep="|")